<a href="https://colab.research.google.com/github/Visionaryofthefuture/HR_PLATFORM/blob/main/FineTuned_Llama3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install huggingface_hub ipython "unsloth[colab] @ git+https://github.com/unslothai/unsloth.git" "unsloth[conda] @ git+https://github.com/unslothai/unsloth.git"


  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-he07gz6t/unsloth_0043b09b549b4baaad726349d8d72c44
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-he07gz6t/unsloth_0043b09b549b4baaad726349d8d72c44
  Resolved https://github.com/unslothai/unsloth.git to commit 27fa021a7bb959a53667dd4e7cdb9598c207aa0d
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.4/102.4 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 41.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 54.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import os
from unsloth import FastLanguageModel
import torch
from trl import SFTTrainer
from transformers import TrainingArguments
from datasets import load_dataset

max_seq_length = 2048
url = "https://huggingface.co/datasets/laion/OIG/resolve/main/unified_chip2.jsonl"
dataset = load_dataset("json", data_files = {"train" : url}, split = "train")

# 2. Load Llama3 model
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = None,
    load_in_4bit = True,
)

# 3 Before training
def generate_text(text):
    inputs = tokenizer(text, return_tensors="pt").to("cuda:0")
    outputs = model.generate(**inputs, max_new_tokens=20)
    print(tokenizer.decode(outputs[0], skip_special_tokens=True))

print("Before training\n")
generate_text("<human>: List the top 5 most popular movies of all time.\n<bot>: ")

# 4. Do model patching and add fast LoRA weights and training
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    use_gradient_checkpointing = True,
    random_state = 3407,
    max_seq_length = max_seq_length,
    use_rslora = False,  # Rank stabilized LoRA
    loftq_config = None, # LoftQ
)

trainer = SFTTrainer(
    model = model,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    tokenizer = tokenizer,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 10,
        max_steps = 60,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        output_dir = "outputs",
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
    ),
)
trainer.train()

# 5. After training
print("\n ######## \nAfter training\n")
generate_text("<human>: List the top 5 most popular movies of all time.\n<bot>: ")


/usr/local/lib/python3.10/dist-packages/unsloth/__init__.py:80: UserWarning: Unsloth: Running `ldconfig /usr/lib64-nvidia` to link CUDA.
  warnings.warn(


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


Generating train split: 0 examples [00:00, ? examples/s]

config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

==((====))==  Unsloth: Fast Llama patching release 2024.5
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.1.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. Xformers = 0.0.22.post7. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


...

Map:   0%|          | 0/210289 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 210,289 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,2.532900
2,2.028200
3,2.503500
4,2.116400
5,2.567300
6,2.320200
7,2.445400
8,1.794500
9,1.735500
10,1.554000


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



 ######## 
After training

<human>: List the top 5 most popular movies of all time.
<bot>: 1. The Shawshank Redemption 2. The Godfather 3. The Dark Knight 


In [7]:
# 5. After training
generate_text("<human>: List The 4 most famous python libraries for data science\n<bot>: ")

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<human>: List The 4 most famous python libraries for data science
<bot>: 1. Pandas: A Python library that provides high-performance, easy-to-use data structures and data


In [9]:
model.save_pretrained("lora_model")
model.save_pretrained_merged("outputs", tokenizer, save_method = "merged_16bit",)
model.push_to_hub_merged("Darkknight12/llama3-8b-oig-unsloth-merged", tokenizer, save_method = "merged_16bit", token = os.environ.get("HF_TOKEN"))


config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

Unsloth: Will remove a cached repo with size 1.2K


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 5.39 out of 12.67 RAM for saving.


100%|██████████| 32/32 [01:52<00:00,  3.50s/it]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving model... This might take 5 minutes for Llama-7b...
Unsloth: Saving outputs/pytorch_model-00001-of-00004.bin...
Unsloth: Saving outputs/pytorch_model-00002-of-00004.bin...
Unsloth: Saving outputs/pytorch_model-00003-of-00004.bin...
Unsloth: Saving outputs/pytorch_model-00004-of-00004.bin...
Done.
Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 5.05 out of 12.67 RAM for saving.


100%|██████████| 32/32 [01:41<00:00,  3.17s/it]


Unsloth: Saving tokenizer...

HfHubHTTPError:  (Request ID: Root=1-66598e8e-3ba59a1223859570281b55a7;a777516b-7cf1-45d3-bd57-e2255dfe26ce)

403 Forbidden: You don't have the rights to create a model under the namespace "Darkknight12".
Cannot access content at: https://huggingface.co/api/repos/create.
If you are trying to create or update content,make sure you have a token with the `write` role.